# references
https://docs.langchain.com/oss/python/langchain/multi-agent/subagents-personal-assistant

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# create Embedding function using Ollama
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="qwen3-embedding:0.6b",
    base_url="https://chat.ilawyer.com.vn/ollama")

In [4]:
# define LLM model first
import os

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="qwen3:30b",
                 api_key=os.getenv("OPEN_WEBUI_OPENAI_KEY"), 
                 base_url=os.getenv("OPEN_WEBUI_OPENAI_URL"),
                 temperature=0.5)

In [4]:
# Create Qdrant client
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
            url = os.getenv("QDRANT_URL"),
            port = 80,
            https = False,
            api_key = os.getenv("QDRANT_API_KEY"),
        )

/tmp/ipykernel_628198/2594988558.py:5: UserWarning: Api key is used with an insecure connection.
  qdrant_client = QdrantClient(


In [ ]:
# create collection in Qdrant
from qdrant_client.http.models import Distance, VectorParams

qdrant_client.create_collection(
    collection_name="20260118_semantic_search_collection",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

In [5]:
# Create the Qdrant vector store
vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name="20260118_semantic_search_collection",
    embedding=embeddings,
)

In [1]:
# define tools
from langchain.tools import tool

@tool
def create_calendar_event(title: str,
                          start_time: str,
                          end_time: str,
                          attendees: list[str],
                          location: str="") -> str:
    """Create a calendar event. Required exact ISO datetime format for start_time and end_time."""
    return f"Event created: {title} from {start_time} to {end_time} at {location} with {len(attendees)} attendees."

@tool
def send_email(
    to: list[str], # email addresses
    subject: str,
    body: str,
    cc: list[str]=[]
) -> str:
    """Send an email via email API. Requires properly formatted addresses."""
    return f"Email sent to {', '.join(to)} with Subject '{subject}'."

@tool
def get_available_time_slots(
    attendees: list[str],
    date: str,  # ISO format date
    duration_minutes: int
) -> list[str]:
    """Get available time slots for given attendees on a specific date."""
    return ["09:00", "14:00", "16:00"]

In [5]:
# create calander agent
from langchain.agents import create_agent

CALENDAR_AGENT_PROMPT = (
    "You are a calendar scheduling assistant. "
    "Parse natural language scheduling requests (e.g, 'next Monday at 3pm')"
    "into proper ISO datetime format. " 
    "Use get_available_time_slots to check available free slots when needed. "
    "Use create_calendar_event to create events. "
    "Always confirm what was scheduled in your final response."
)

calendar_agent = create_agent(
    model,
    tools=[create_calendar_event, get_available_time_slots],
    system_prompt=CALENDAR_AGENT_PROMPT
)

In [6]:
# first test calendar agent
# query = "Schedule a meeting with John Doe and Jane Smith next Tuesday at 10am for 1 hour about the project update."
query = "Schedule a team meeting next Tuesday at 2pm for 1 hour"

for step in calendar_agent.stream({"messages": [{"role": "user", "content": query}]}):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_available_time_slots (call_ii6txgs6)
 Call ID: call_ii6txgs6
  Args:
    attendees: ['Team']
    date: 2023-10-17
    duration_minutes: 60
================================= Tool Message =================================
Name: get_available_time_slots

["09:00", "14:00", "16:00"]
================================== Ai Message ==================================
Tool Calls:
  create_calendar_event (call_6ueeyp5h)
 Call ID: call_6ueeyp5h
  Args:
    attendees: ['Team']
    end_time: 2023-10-17T15:00:00
    start_time: 2023-10-17T14:00:00
    title: Team Meeting
================================= Tool Message =================================
Name: create_calendar_event

Event created: Team Meeting from 2023-10-17T14:00:00 to 2023-10-17T15:00:00 at  with 1 attendees.
================================== Ai Message ==================================

Your team meeting has been scheduled for **Tuesd

In [7]:
# create an email agent
EMAIL_AGENT_PROMPT = (
    "You are an email assistant. "
    "compose professional emails based on natural language requests. "
    "Extract recipients information and craft approriate subject lines and bodies text."
    "Use send_email tool to send the email."
    "Always confirm what was sent in your final response."
)
email_agent = create_agent(
    model,
    tools=[send_email],
    system_prompt=EMAIL_AGENT_PROMPT
)

In [8]:
# test email agent
email_query = "Send the design team a reminder about reviewing the new mockups"

for step in email_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  send_email (call_rvly10fr)
 Call ID: call_rvly10fr
  Args:
    body: Hello team,

This email confirms our team meeting scheduled for next Tuesday at 2:00 PM, lasting one hour. Please ensure your availability and prepare any necessary updates.

Best regards,
[Your Name]
    subject: Team Meeting Scheduled for Next Tuesday at 2:00 PM
    to: ['team@company.com']
================================= Tool Message =================================
Name: send_email

Email sent to team@company.com with Subject 'Team Meeting Scheduled for Next Tuesday at 2:00 PM'.
================================== Ai Message ==================================

The email has been successfully sent to team@company.com with the subject line:  
**"Team Meeting Scheduled for Next Tuesday at 2:00 PM"**.


In [10]:
# wrap both agents as tools for supervisor agent
@tool
def schedule_event(request: str) -> str:
    """Schedule calendar events using natural language.

    Use this when the user wants to create, modify, or check calendar appointments.
    Handles date/time parsing, availability checking, and event creation.

    Input: Natural language scheduling request (e.g., 'meeting with design team
    next Tuesday at 2pm')
    """
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text


@tool
def manage_email(request: str) -> str:
    """Send emails using natural language.

    Use this when the user wants to send notifications, reminders, or any email
    communication. Handles recipient extraction, subject generation, and email
    composition.

    Input: Natural language email request (e.g., 'send them a reminder about
    the meeting')
    """
    result = email_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text

In [11]:
# create supervisor agent with tools
SUPERVISOR_AGENT_PROMPT = (
    "You are a personal assistant managing both calendar and email tasks. "
    "Decide whether to use the schedule_event tool for scheduling needs "
    "or the manage_email tool for email communications based on user requests and coordinate the results."
    "When a request involves multiple actions, use multiple tools in sequence"
)

supervisor_agent = create_agent(
    model,
    tools=[schedule_event, manage_email],
    system_prompt=SUPERVISOR_AGENT_PROMPT,
)

In [12]:
# Example 1: Simple single-domain request

query = "Schedule a team standup for tomorrow at 9am"

for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_wgb68n9p)
 Call ID: call_wgb68n9p
  Args:
    request: Schedule a team standup for tomorrow at 9am
================================= Tool Message =================================
Name: schedule_event

The team standup has been successfully scheduled for **tomorrow, October 6, 2023, from 9:00 AM to 9:30 AM** with the team as attendees. The event is confirmed in the calendar.
================================== Ai Message ==================================

The team standup has been successfully scheduled for **tomorrow, October 6, 2023, from 9:00 AM to 9:30 AM** with the team as attendees. The event is confirmed in your calendar. Let me know if you'd like to send a reminder email to the team!


In [13]:
# Example 2: Complex multi-domain request

query = (
    "Schedule a meeting with the design team next Tuesday at 2pm for 1 hour, "
    "and send them an email reminder about reviewing the new mockups."
)

for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_n1xt0bg1)
 Call ID: call_n1xt0bg1
  Args:
    request: meeting with design team next Tuesday at 2pm for 1 hour
  manage_email (call_tlx8lqhe)
 Call ID: call_tlx8lqhe
  Args:
    request: send them a reminder about reviewing the new mockups
================================= Tool Message =================================
Name: manage_email

Email sent to design-team@example.com with subject "Reminder: Review New Mockups".
================================= Tool Message =================================
Name: schedule_event

Your event "meeting with design team" has been scheduled successfully for **Tuesday, October 3, 2023, from 2:00 PM to 3:00 PM** with the design team.
================================== Ai Message ==================================

Your meeting with the design team has been scheduled for **Tuesday, October 3, 2023, from 2:00 PM to 3:00 PM**. The email rem

In [14]:
## Add human in the loop
# create calander agent
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware 
from langgraph.checkpoint.memory import InMemorySaver 

CALENDAR_AGENT_PROMPT = (
    "You are a calendar scheduling assistant. "
    "Parse natural language scheduling requests (e.g, 'next Monday at 3pm')"
    "into proper ISO datetime format. " 
    "Use get_available_time_slots to check available free slots when needed. "
    "Use create_calendar_event to create events. "
    "Always confirm what was scheduled in your final response."
)

calendar_agent = create_agent(
    model,
    tools=[create_calendar_event, get_available_time_slots],
    system_prompt=CALENDAR_AGENT_PROMPT,
    middleware=[HumanInTheLoopMiddleware(interrupt_on={"create_calendar_event": True},
                                         description_prefix="Calendar event pending approval")]
)

# create an email agent
EMAIL_AGENT_PROMPT = (
    "You are an email assistant. "
    "compose professional emails based on natural language requests. "
    "Extract recipients information and craft approriate subject lines and bodies text."
    "Use send_email tool to send the email."
    "Always confirm what was sent in your final response."
)

email_agent = create_agent(
    model,
    tools=[send_email],
    system_prompt=EMAIL_AGENT_PROMPT,
    middleware=[ 
        HumanInTheLoopMiddleware( 
            interrupt_on={"send_email": True}, 
            description_prefix="Outbound email pending approval", 
        ), 
    ],
)

# create supervisor agent with tools
SUPERVISOR_AGENT_PROMPT = (
    "You are a personal assistant managing both calendar and email tasks. "
    "Decide whether to use the schedule_event tool for scheduling needs "
    "or the manage_email tool for email communications based on user requests and coordinate the results."
    "When a request involves multiple actions, use multiple tools in sequence"
)

supervisor_agent = create_agent(
    model,
    tools=[schedule_event, manage_email],
    system_prompt=SUPERVISOR_AGENT_PROMPT,
    checkpointer=InMemorySaver(), 
)

In [15]:
# re-test query with human in the loop enabled
query = (
    "Schedule a meeting with the design team next Tuesday at 2pm for 1 hour, "
    "and send them an email reminder about reviewing the new mockups."
)

config = {"configurable": {"thread_id": "6"}}

interrupts = []

for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    config,
):
    for update in step.values():
        if isinstance(update, dict):
            for message in update.get("messages", []):
                message.pretty_print()
        else:
            interrupt_ = update[0]
            interrupts.append(interrupt_)
            print(f"\nINTERRUPTED: {interrupt_.id}")

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_i7d4htib)
 Call ID: call_i7d4htib
  Args:
    request: Schedule a meeting with the design team next Tuesday at 2pm for 1 hour
  manage_email (call_wfdnya93)
 Call ID: call_wfdnya93
  Args:
    request: send them an email reminder about reviewing the new mockups
================================= Tool Message =================================
Name: manage_email

I need the recipient email addresses to send this reminder. Could you please provide the email addresses for the recipients who need to review the new mockups?

INTERRUPTED: d6e4a6837ff0824b0e4c8ed3d5d76d19


In [28]:
# explore interrupts
for interrupt_ in interrupts:
    for request in interrupt_.value["action_requests"]:
        print(f"INTERRUPTED: {interrupt_.id}")
        print(f"{request['description']}\n")

In [27]:
for interrupt_ in interrupts:
    if interrupt_.id == "d6e4a6837ff0824b0e4c8ed3d5d76d19":
        edited_action = interrupt_.value["action_requests"][0].copy()
        print(edited_action)

In [29]:
# approve, edit email and resume
from langgraph.types import Command 

resume = {}
for interrupt_ in interrupts:
    if interrupt_.id == "bd7adf47100d22dec96d37ea888a7eda":
        # Edit email
        edited_action = interrupt_.value["action_requests"][0].copy()
        edited_action["args"]["attendees"] = "Mockups team"
        resume[interrupt_.id] = {
            "decisions": [{"type": "edit", "edited_action": edited_action}]
        }
    else:
        resume[interrupt_.id] = {"decisions": [{"type": "approve"}]}

interrupts = []
for step in supervisor_agent.stream(
    Command(resume=resume), 
    config,
):
    for update in step.values():
        if isinstance(update, dict):
            for message in update.get("messages", []):
                message.pretty_print()
        else:
            interrupt_ = update[0]
            interrupts.append(interrupt_)
            print(f"\nINTERRUPTED: {interrupt_.id}")

# Pass additional conversational context to sub-agents

In [ ]:
from langchain.tools import tool, ToolRuntime

@tool
def schedule_event(
    request: str,
    runtime: ToolRuntime
) -> str:
    """Schedule calendar events using natural language."""
    # Customize context received by sub-agent
    original_user_message = next(
        message for message in runtime.state["messages"]
        if message.type == "human"
    )

    # Construct a prompt that includes the original user message and pass to calendar agent
    prompt = (
        "You are assisting with the following user inquiry:\n\n"
        f"{original_user_message.text}\n\n"
        "You are tasked with the following sub-request:\n\n"
        f"{request}"
    )
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": prompt}],
    })
    return result["messages"][-1].text

# Control what supervisor receives

In [ ]:
import json

@tool
def schedule_event(request: str) -> str:
    """Schedule calendar events using natural language."""
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })

    # Option 1: Return just the confirmation message
    return result["messages"][-1].text

    # Option 2: Return structured data
    # return json.dumps({
    #     "status": "success",
    #     "event_id": "evt_123",
    #     "summary": result["messages"][-1].text
    # })